#VISIÓN POR COMPUTADOR

*MARTA GORRAIZ,
IRENE GONZÁLEZ,
IKER MARCELO*

In [ ]:
!pip install mahotas

##CONECTAR CON DRIVE

Conestarse a drive para poder acceder al dataser

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Realizar los import necesarios

In [ ]:
import os
import shutil
import cv2
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import mahotas as mh

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model


##DATA AUGMENTATION


In [ ]:
from PIL import Image, ImageEnhance
import os
from pathlib import Path

# Lista de rutas a las carpetas con imágenes
directories_to_process = [
    '/content/drive/MyDrive/dataset/platano/platano_bien',
    '/content/drive/MyDrive/dataset/platano/platano_defecto',
    '/content/drive/MyDrive/dataset/fresa/fresa_bien',
    '/content/drive/MyDrive/dataset/fresa/fresa_defecto',
    '/content/drive/MyDrive/dataset/limon/limon_bien',
    '/content/drive/MyDrive/dataset/limon/limon_defecto',
    '/content/drive/MyDrive/dataset/uva/uva_bien',
    '/content/drive/MyDrive/dataset/uva/uva_defecto',
    '/content/drive/MyDrive/dataset/pera/pera_defecto',
    '/content/drive/MyDrive/dataset/pera/pera_bien'

]

# Directorio base para guardar las imágenes con brillo modificado
output_base_dir = '/content/drive/MyDrive/dataset_modificado/'

# Niveles de brillo a aplicar
brightness_levels = [0.5, 1.5]  # Brillo reducido y aumentado

# Crear carpetas de salida y procesar imágenes
for input_dir in directories_to_process:
    for brightness in brightness_levels:
        # Crear ruta de salida
        output_dir = Path(output_base_dir) / Path(input_dir).name

        if not output_dir.exists():
            output_dir.mkdir(parents=True, exist_ok=True)
            print(f"Carpeta de salida creada: {output_dir}")
        else:
            print(f"Carpeta de salida ya existe: {output_dir}")

        # Procesar cada imagen en la carpeta
        for image_name in os.listdir(input_dir):
            input_path = os.path.join(input_dir, image_name)

            # Verificar si es un archivo de imagen
            if os.path.isfile(input_path) and image_name.lower().endswith(('png', 'jpg', 'jpeg')):
                try:
                    # Abrir imagen
                    img = Image.open(input_path)

                    # Modificar el brillo
                    enhancer = ImageEnhance.Brightness(img)
                    img_enhanced = enhancer.enhance(brightness)

                    # Guardar la imagen modificada
                    base_name, ext = os.path.splitext(image_name)
                    output_path = os.path.join(output_dir, f"{base_name}_brillo_{brightness}{ext}")
                    img_enhanced.save(output_path)
                except Exception as e:
                    print(f"Error procesando la imagen {input_path}: {e}")

print("Procesamiento completado.")


##PROCESAR LAS IMAGENES



Definir las rutas de las carpetas a usar con el data augmentation



In [ ]:
directories_to_process = [
    '/content/drive/MyDrive/dataset_modificado/platano/platano_bien',
    '/content/drive/MyDrive/dataset_modificado/platano/platano_defecto',
    '/content/drive/MyDrive/dataset_modificado/fresa/fresa_bien',
    '/content/drive/MyDrive/dataset_modificado/fresa/fresa_defecto',
    '/content/drive/MyDrive/dataset_modificado/limon/limon_bien',
    '/content/drive/MyDrive/dataset_modificado/limon/limon_defecto',
    '/content/drive/MyDrive/dataset_modificado/uva/uva_bien',
     '/content/drive/MyDrive/dataset_modificado/pera/pera_defecto',
    '/content/drive/MyDrive/dataset_modificado/pera/pera_bien',
    '/content/drive/MyDrive/dataset/platano/platano_bien',
    '/content/drive/MyDrive/dataset/platano/platano_defecto',
    '/content/drive/MyDrive/dataset/fresa/fresa_bien',
    '/content/drive/MyDrive/dataset/fresa/fresa_defecto',
    '/content/drive/MyDrive/dataset/limon/limon_bien',
    '/content/drive/MyDrive/dataset/limon/limon_defecto',
    '/content/drive/MyDrive/dataset/uva/uva_bien',
    '/content/drive/MyDrive/dataset/pera/pera_defecto',
    '/content/drive/MyDrive/dataset/pera/pera_bien'
]

Código para extraer las características de las imagenes

In [ ]:
import cv2
import numpy as np
import os
import math
import pandas as pd
import mahotas as mh


# Ajustar brillo y contraste de la imagen
def adjust_brightness_contrast(image, alpha=1, beta=70):
    return cv2.convertScaleAbs(image, alpha=alpha, beta=beta)


# Procesar imagen
def process_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"No se pudo leer la imagen: {image_path}")
        return None, None, None, None
    image = cv2.resize(image, (600, 400))

    bright_image = adjust_brightness_contrast(image)
    # Filtro gaussiano y canny
    blurred = cv2.GaussianBlur(bright_image, (5, 5), 0)
    edges = cv2.Canny(blurred, 100, 200)

    # Encontrar contornos
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    return bright_image, edges, None, contours

# Obtener los rangos de los cinco colores más predominantes
def get_color_ranges(image, contours):
    if len(contours) == 0:
        return None


    mask = np.zeros(image.shape[:2], dtype=np.uint8)

    # Procesar cada contorno
    for cnt in contours:
        epsilon = 0.01 * cv2.arcLength(cnt, True)
        hull = cv2.convexHull(cnt)
        cv2.drawContours(mask, [hull], -1, (255), thickness=cv2.FILLED)

    # Aplicar máscara y convertir a HSV
    masked_image = cv2.bitwise_and(image, image, mask=mask)
    hsv_image = cv2.cvtColor(masked_image, cv2.COLOR_BGR2HSV)

    # Filtrar el color blanco y tonos claros
    lower_bound = np.array([0, 50, 0])
    upper_bound = np.array([180, 255, 230])
    filtered_mask = cv2.inRange(hsv_image, lower_bound, upper_bound)

    # Excluir el color azul
    blue_lower_bound = np.array([100, 50, 0])
    blue_upper_bound = np.array([140, 255, 255])
    blue_mask = cv2.inRange(hsv_image, blue_lower_bound, blue_upper_bound)
    filtered_mask = cv2.bitwise_and(filtered_mask, cv2.bitwise_not(blue_mask))

    # Aplicar la máscara filtrada
    filtered_hsv_image = cv2.bitwise_and(hsv_image, hsv_image, mask=filtered_mask)

    # Histograma del canal H con la nueva máscara
    h_channel = filtered_hsv_image[:, :, 0]
    hist = cv2.calcHist([h_channel], [0], filtered_mask, [180], [0, 180])

    # Obtener los tonos dominantes
    dominant_hues = np.argsort(hist.flatten())[-5:]
    ranges = [(int(h), int(h + 10)) for h in dominant_hues]

    return ranges

# calcular rugosidad
def calculate_simple_rugosity(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rugosity = np.var(gray)

    return rugosity

# Calcular características de forma
def calculate_shape_features(contours):

    if len(contours) == 0:
        return None
    largest_contour = max(contours, key=cv2.contourArea)

    # Área y perímetro
    area = cv2.contourArea(largest_contour)
    perimeter = cv2.arcLength(largest_contour, True)

    # Relación de aspecto y dimensiones
    x, y, w, h = cv2.boundingRect(largest_contour)
    aspect_ratio = float(w) / h if h != 0 else 0

    # Altura y Anchura
    height = h
    width = w

    # Circularidad
    circularity = (4 * math.pi * area) / (perimeter ** 2) if perimeter != 0 else 0

    # Momentos Hu
    moments = cv2.moments(largest_contour)
    hu_moments = cv2.HuMoments(moments).flatten()

    # Coordenadas normalizadas del contorno
    normalized_contour = largest_contour - [x, y]
    normalized_contour = normalized_contour / [w, h]

    # Calcular estadisticas de las coordenadas normalizadas
    mean_coord = np.mean(normalized_contour, axis=0).flatten()
    std_coord = np.std(normalized_contour, axis=0).flatten()

    shape_features = {
        'area': area,
        'perimeter': perimeter,
        'aspect_ratio': aspect_ratio,
        'height': height,
        'width': width,
        'circularity': circularity,
        'hu_moments': hu_moments.tolist(),
        'mean_normalized_contour_x': mean_coord[0],
        'mean_normalized_contour_y': mean_coord[1],
        'std_normalized_contour_x': std_coord[0],
        'std_normalized_contour_y': std_coord[1]
    }

    return shape_features


# Función para calcular características de GLCM usando Mahotas
def calculate_glcm_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Calcular las características GLCM usando haralick
    haralick_features = mh.features.texture.haralick(gray, distance=1).mean(axis=0)

    # Extraer las características necesarias
    contrast = haralick_features[1]
    correlation = haralick_features[2]
    energy = haralick_features[8]
    homogeneity = haralick_features[4]

    return contrast, correlation, energy, homogeneity



# Recorrer cada directorio especificado
for dir_path in directories_to_process:
    path_parts = dir_path.strip('/').split('/')
    fruit_type = path_parts[-2]  # Nombre de la fruta
    condition = path_parts[-1]

    category = condition

    # Obtener todas las imágenes en la carpeta
    for image_name in os.listdir(dir_path):
        image_path = os.path.join(dir_path, image_name)
        if image_path.endswith(('.jpg', '.JPG', '.jpeg', '.png', '.HEIC')):
            # Procesar la imagen y extraer características
            original_image, edges_image, _, contours = process_image(image_path)
            if original_image is None:
                continue
            color_ranges = get_color_ranges(original_image, contours)
            rugosity = calculate_simple_rugosity(original_image)
            shape_features = calculate_shape_features(contours)
            number_of_contours = len(contours)

            # Calcular características GLCM
            contrast, correlation, energy, homogeneity = calculate_glcm_features(original_image)

            # Crear un diccionario con todas las características
            if shape_features:
                features = {
                    'fruit_type': fruit_type,
                    'category': category,
                    'image_name': image_name,
                    'rugosity': rugosity,
                    'number_of_contours': number_of_contours,
                    'area': shape_features['area'],
                    'perimeter': shape_features['perimeter'],
                    'aspect_ratio': shape_features['aspect_ratio'],
                    'height': shape_features['height'],
                    'width': shape_features['width'],
                    'circularity': shape_features['circularity'],
                    'mean_normalized_contour_x': shape_features['mean_normalized_contour_x'],
                    'mean_normalized_contour_y': shape_features['mean_normalized_contour_y'],
                    'std_normalized_contour_x': shape_features['std_normalized_contour_x'],
                    'std_normalized_contour_y': shape_features['std_normalized_contour_y'],
                    'contrast': contrast,
                    'correlation': correlation,
                    'energy': energy,
                    'homogeneity': homogeneity
                }

                for i, hu_moment in enumerate(shape_features['hu_moments']):
                    features[f'hu_moment_{i+1}'] = hu_moment

                for i, h_range in enumerate(color_ranges):
                    features[f'hue_range_{i+1}_start'] = h_range[0]
                    features[f'hue_range_{i+1}_end'] = h_range[1]


                # Añadir las características a la lista
                data.append(features)
            else:
                print(f"No se pudieron extraer características de forma para la imagen {image_name}")
        else:
            print(f"Archivo no es una imagen: {image_name}")

# Convertir la lista de características en un DataFrame
df = pd.DataFrame(data)

# Mostrar las primeras filas del DataFrame
df.head(10)

No se pudieron extraer características de forma para la imagen IMG_20241023_202549.jpg
No se pudo leer la imagen: /content/drive/MyDrive/dataset/limon/limon_bien/IMG_7712.HEIC


,fruit_type,category,image_name,rugosity,number_of_contours,area,perimeter,aspect_ratio,height,width,...,hue_range_1_start,hue_range_1_end,hue_range_2_start,hue_range_2_end,hue_range_3_start,hue_range_3_end,hue_range_4_start,hue_range_4_end,hue_range_5_start,hue_range_5_end
0,platano,platano_bien,IMG_20241029_130154.jpg,253.041027,30,220.5,1867.138293,1.883929,224,422,...,15,25,16,26,17,27,18,28,19,29
1,platano,platano_bien,IMG_20241029_130136.jpg,300.392883,26,112.0,618.666081,1.041176,170,177,...,15,25,14,24,11,21,12,22,13,23
2,platano,platano_bien,IMG_20241029_130112.jpg,296.440448,25,188.0,1898.846882,1.082840,338,366,...,21,31,17,27,20,30,19,29,18,28
3,platano,platano_bien,IMG_20241029_130101.jpg,286.637281,10,240.0,2246.944421,1.931174,247,477,...,16,26,19,29,20,30,18,28,17,27
4,platano,platano_bien,IMG_20241029_130048.jpg,342.070948,14,151.5,803.702663,1.129707,239,270,...,17,27,15,25,30,40,29,39,16,26
5,platano,platano_bien,IMG_20241029_130029.jpg,237.184075,18,328.0,1582.459143,1.249057,265,331,...,19,29,15,25,18,28,17,27,16,26
6,platano,platano_bien,IMG_20241029_130015.jpg,228.442043,14,50.0,261.705624,1.584615,65,103,...,19,29,15,25,18,28,17,27,16,26
7,platano,platano_bien,IMG_20241029_130003_BURST1.jpg,224.683373,15,192.0,1432.645872,0.961644,365,351,...,15,25,19,29,17,27,16,26,18,28
8,platano,platano_bien,IMG_20241029_125955.jpg,211.725586,17,154.5,1505.207275,2.339367,221,517,...,21,31,19,29,20,30,22,32,23,33
9,platano,platano_bien,IMG_20241029_125946.jpg,402.788678,15,227.5,1634.368742,2.281553,206,470,...,12,22,17,27,13,23,16,26,11,21


##ALMACENAR INFORMACIÓN

Se almacena la información en un csv y se especifica la ruta donde se guarda.

In [ ]:
df.to_csv('/content/drive/MyDrive/dataset_features_bien.csv', index=False)